In [62]:
from bs4 import BeautifulSoup, SoupStrainer
import cchardet
import lxml
import numpy as np
import pandas as pd
import requests

In [128]:

AGENT = {"User-Agent": "Edge"}

def scrape_game_info(pg_num: int=0, agent: dict=AGENT) -> tuple[list]:
    """
    """
    ROOT = "https://www.metacritic.com"
    SELECT_CRITERIA = """
        a.title,
        .platform span.data,
        .clamp-metascore div,
        .clamp-userscore div
    """
    pg_url = ROOT + "/browse/games/score/metascore/all?page=" + str(pg_num)
    all_pg = requests.get(pg_url, headers=agent)
    game_cells = SoupStrainer("td", class_="clamp-summary-wrap")
    games_html = BeautifulSoup(all_pg.content, "lxml", parse_only=game_cells)
    games_info = games_html.select(SELECT_CRITERIA)
    titles = [g.get_text() for g in games_info[::4]]
    game_urls = [ROOT + g["href"] for g in games_info[::4]]
    platforms = [g.get_text(strip=True) for g in games_info[1::4]]
    metascores = np.fromiter((g.get_text() for g in games_info[2::4]), "B")
    userscores = np.fromiter((g.get_text() for g in games_info[3::4]), "f")
    return titles, platforms, metascores, userscores, game_urls
titles, platforms, metascores, userscores, game_urls  = scrape_game_info()

In [141]:
def get_genre(game_urls: list[str]) -> list[list[str]]:
    """
    """
    releases = []
    genres = []
    for url in game_urls:
        game_pg = requests.get(url, headers=AGENT)
        body = SoupStrainer("div", class_="left")
        game_html = BeautifulSoup(game_pg.content, "lxml", parse_only=body)
        genre_info = game_html.select("li.summary_detail.product_genre .data")
        release = game_html.select_one("li.summary_detail.release_data .data")
        genre_lst = set([genre.get_text() for genre in genre_info])
        genres.append(genre_lst)
        releases.append(release.get_text())
    return genres, releases

genres, releases = get_genre(game_urls)

In [ ]:
def get_critic_scores(game_urls: list[str]):
    """
    """
    SUFFIX = "\critic-reviews"
    critic_scores = []
    for url in game_urls:
        critic_pg = requests.get(url + SUFFIX, headers=AGENT)
        
    return

In [156]:
data_dict = {
    "Title": titles,
    "Platform": platforms,
    "Release Date": releases,
    "Metascore": metascores,
    "Userscore": userscores,
    "Genres": genres,
}

df = pd.DataFrame(data_dict)
f = lambda x: pd.Series({f"Genre_{i}": list(x.Genres)[i] for i, _ in enumerate(x.Genres)})
pd.concat([df.drop("Genres", axis=1), df.apply(f, axis=1)], axis=1)

,Title,Platform,Release Date,Metascore,Userscore,Genre_0,Genre_1,Genre_2,Genre_3,Genre_4,Genre_5
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"Nov 23, 1998",99,9.1,Action Adventure,Fantasy,NaN,NaN,NaN,NaN
1,Tony Hawk's Pro Skater 2,PlayStation,"Sep 20, 2000",98,7.5,Sports,Alternative,Skateboarding,NaN,NaN,NaN
2,Grand Theft Auto IV,PlayStation 3,"Apr 29, 2008",98,7.8,Open-World,Action Adventure,Modern,NaN,NaN,NaN
3,SoulCalibur,Dreamcast,"Sep 8, 1999",98,8.4,Action,Fighting,3D,NaN,NaN,NaN
4,Grand Theft Auto IV,Xbox 360,"Apr 29, 2008",98,8.0,Open-World,Action Adventure,Modern,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,Sid Meier's Civilization II,PC,"Feb 29, 1996",94,8.7,4X,General,Strategy,Historic,Turn-Based,NaN
96,Quake,PC,"Jun 22, 1996",94,8.7,Shooter,Action,First-Person,Sci-Fi,Arcade,NaN
97,Call of Duty 4: Modern Warfare,Xbox 360,"Nov 5, 2007",94,8.5,Shooter,Action,Arcade,First-Person,Modern,NaN
98,BioShock Infinite,PC,"Mar 25, 2013",94,8.6,Shooter,Action,First-Person,Sci-Fi,Arcade,NaN


In [ ]:
#while True:
    #pg_num = 0
    #fetch_game_urls(pg_num)